In [41]:
import seaborn as sns
import pandas as pd
import numpy as np

In [42]:
# Load dataset
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [43]:
# Inspect the dataset
titanic.info()
titanic.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [44]:
# Drop variable deck (too many missing values)
clean_titanic = titanic.drop(columns=["deck"])

In [45]:
# Fill embarked with mode
clean_titanic["embarked"] = clean_titanic["embarked"].fillna(clean_titanic["embarked"].mode()[0])

# Drop embark_town (similar info to embarked)
clean_titanic = clean_titanic.drop(columns=["embark_town"])

clean_titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,no,True


In [46]:
# Use random imputation of random distribution to fill missing age values

np.random.seed(42)  # ensures the random numbers are the same each run

In [47]:
# Compute the mean, std, and missing count 

mean_age = clean_titanic["age"].mean()
std_age = clean_titanic["age"].std()
missing_count = clean_titanic["age"].isnull().sum()

print(f"Mean Age: {mean_age:.2f}, Std Dev: {std_age:.2f}, Missing: {missing_count}")

Mean Age: 29.70, Std Dev: 14.53, Missing: 177


In [48]:
# Determine the number of missing Age values to fill randomly (10% of all missing values)

percent_random = 0.1  # 10% of missing values
fill_count = int(missing_count * percent_random)

In [49]:
# Get indices of missing ages
missing_indices = clean_titanic[clean_titanic["age"].isnull()].index

# Randomly select 10% of them
random_fill_indices = np.random.choice(missing_indices, size=fill_count, replace=False)

In [50]:
# Generate ages from normal distribution (clip at 0 so no negatives)
random_ages = np.random.normal(mean_age, std_age, size=fill_count).clip(0)

# Fill selected rows
clean_titanic.loc[random_fill_indices, "age"] = random_ages

In [51]:
# Fill remaining ages with the median
clean_titanic["age"] = clean_titanic["age"].fillna(clean_titanic["age"].median())

In [52]:
# Standardize gender to 'M' and 'F'
clean_titanic["sex"] = clean_titanic["sex"].map({"male": "M", "female": "F"})

In [53]:
# Rename variable names

clean_titanic.rename(columns={
    "survived": "Survived",
    "pclass": "Class",
    "sex": "Sex",
    "age": "Age",
    "sibsp": "SiblingsSpouses",
    "parch": "ParentsChildren",
    "fare": "Fare"
}, inplace=True)

# Keep the variables useful for analysis
clean_titanic = clean_titanic[["Survived", "Class", "Sex", "Age", "Fare", "SiblingsSpouses", "ParentsChildren", "embarked"]]
clean_titanic.head()


,Survived,Class,Sex,Age,Fare,SiblingsSpouses,ParentsChildren,embarked
0,0,3,M,22.0,7.2500,1,0,S
1,1,1,F,38.0,71.2833,1,0,C
2,1,3,F,26.0,7.9250,0,0,S
3,1,1,F,35.0,53.1000,1,0,S
4,0,3,M,35.0,8.0500,0,0,S


In [54]:
# Save cleaned Titanic dataset to CSV
clean_titanic.to_csv("clean_titanic.csv", index=False)